# MPS canonical forms 

In [1]:
%load_ext autoreload
%autoreload 2


In [2]:


import numpy as np
from tensornetwork import ncon
import logging



logging.basicConfig()
logger = logging.getLogger('Something')
logger.setLevel(logging.DEBUG)


import copy 

In [3]:

import myMPSstuff as mps
import myMPOstuff as mpo

import isingMPO as isi
import applMPOMPS as mpomps
from matrixUtils import checkIdMatrix

import myEnvironmentsMPO as envs



In [4]:

test2 = mps.myMPS(mps.randMPS(chi=100))
print(f"Entropies=  {test2.getEntropies()}")


Entropies=  [0.0, 2.3667490570580708e-05, 1.626997496883383e-06, 2.566595653706712e-07, 1.6785939494629589e-07, 2.356137399456464e-07, 9.54094761996486e-07, 9.411157407699243e-08, 8.541148761398331e-07, 2.4708670878542946e-05, 0.0]


In [5]:

testMPO = mpo.myMPO(isi.IsingMPO(LL=10, J = 1, g = 0.4))


In [6]:

#mpstest = np.random.rand(4,7,3,7)

test = mps.myMPS()

print(test.chis)
print([np.shape(s) for s in test.MPS])

print(test.getNormSlow())
print(test.getNorm())



[1, 20, 20, 20, 20, 20, 20, 1]
[(1, 20, 2), (20, 20, 2), (20, 20, 2), (20, 20, 2), (20, 20, 2), (20, 20, 2), (20, 1, 2)]
(3534725124498788.5+0.0458984375j)
3534725124498788.0


In [7]:
test = mps.myMPS(mps.plusState(10))

print(test.chis)
print([np.shape(s) for s in test.MPS])

# the state should be normalized already 
print(test.getNormSlow())
print(test.getNorm())

test.bringCan()
print(test.getEntropies())  # should be zero for a prod state



[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[(1, 1, 2), (1, 1, 2), (1, 1, 2), (1, 1, 2), (1, 1, 2), (1, 1, 2), (1, 1, 2), (1, 1, 2), (1, 1, 2), (1, 1, 2)]
0.9999999999999978
0.9999999999999984
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [8]:
test = mps.myMPS()
print(test.getNormSlow())

print(test.getNorm())
test.bringCan()

(3534725124498788.5+0.0458984375j)
3534725124498788.0


'R'

In [9]:


test3 = mps.myMPS(mps.randMPS(LL=20, chi=200))
print(f"Entropies=  {test3.getEntropies()}")

testMPO = mpo.myMPO()
#print(testMPO.getIndices())



Entropies=  [0.0, 5.939512810731911e-06, 1.6622680821985763e-07, 8.131511584464328e-09, 2.912752506359213e-08, 3.729321308612988e-08, 3.4878182404457804e-08, 3.0556395350425725e-08, 7.563091905336443e-08, 2.462029304588088e-07, 1.9108455538284102e-08, 2.2688003065365525e-08, 3.6125910892548514e-08, 5.874845717310322e-08, 1.5632923214608612e-08, 2.2562467675941242e-08, 4.480089139257817e-08, 2.465136216867887e-08, 8.140148379821287e-08, 9.269116515088067e-06, 0.0]


## Check that we're actually in proper L-R canon forms by contracting a bit and see if we recover identity matrices and normalizations 

In [10]:
mymps3 = mps.myMPS(mps.randMPS(10,chi=30))

mymps3.bringCan(epsTrunc=1e-6)
mymps3.set_form(mode='L')


# So now the MPS should be normalized and in  LEFT canonical form 
# Check by contracting some A's 

print(f"Norm={mymps3.getNorm()}")

mp = mymps3.MPS
sv = mymps3.SV
svinv = mymps3.SVinv


[ checkIdMatrix(ncon([m,np.conj(m)],[[1,-1,2],[1,-2,2]])) for m in mymps3.MPS ]

# Looks good 




Norm=1.0000000000000002
identity, size = 2
identity, size = 4
identity, size = 4
identity, size = 4
identity, size = 4
identity, size = 4
identity, size = 4
identity, size = 4
identity, size = 2
identity, size = 1


[True, True, True, True, True, True, True, True, True, True]

## Try computing an expectation value, first the "hard" way (ncon the whole thing), then using the right-canonical expression

In [11]:
# Try to do it a bit better, define separately psi and psi*
# and compute the norm 

testL = 12
testd = 2
testChi = 8

mps1 = mps.randMPS(testL,chi=testChi, d=testd)

mypsi = mps.myMPS(mps1)
#mypsic = mps.myMPS([np.conj(m) for m in mps1])

# make two copies for later messing around 
mypsic1 = copy.deepcopy(mypsi)
mypsic1conj = mps.myMPS([np.conj(m) for m in mypsic1.MPS])

mypsic2 = copy.deepcopy(mypsi)

print(type(mypsi),type(mypsic1))

# Now define the operator to exp-value,
# say we wanna compute <sigma_x> 
op1 = np.array([[0,1.],[1.,0]])

# at site r = 3 

print("Exp value the hard way: ")

insertjj = 3 
tryind = [[1,3,2]]
tryind.append([1,4,2])

for jj in range(1,testL):
    tj = 3*jj
    if jj == insertjj: # horrible hack 
        tryind.append([tj,tj+3,tj+2])
        tryind.append([tj+2,9999])
        tryind.append([tj+1,tj+4,9999])
    else:
        tryind.append([tj,tj+3,tj+2])
        tryind.append([tj+1,tj+4,tj+2])

tryind[-2][1] = tryind[-1][1]

opList = []
for jj, m in enumerate(mypsi.MPS):
    if jj == insertjj:
        opList.append(m)
        opList.append(op1)
        opList.append(m.conj())
    else:
        opList.append(m)
        opList.append(m.conj())

ev = ncon(opList,tryind)

#print(tryind )
print(np.real_if_close(ev/mypsi.getNormSlow() ))
#print(np.real_if_close(ev/mypsi.getNorm() ))

print("Second way: apply the op to the relevant site, then compute the overlap with itself")
mypsic1.MPS[3] = ncon([op1,mypsic1.MPS[3]],[[-3,1],[-1,-2,1]])
expvaloverlap = mypsic1.overlap(mypsic1conj) 
print(np.real_if_close(expvaloverlap/mypsic1.getNormSlow() ))

print(" Now compute it with the canonical form ")

mypsic2.set_form('R')

conTen = [np.diag(mypsic2.SV[3]),np.diag(mypsic2.SV[3]),mypsic2.MPS[3],np.conj(mypsic2.MPS[3]),op1]
conIdx = [[1,2],[1,3],[3,5,4],[2,5,6],[4,6]]

[print(np.shape(elem)) for elem in conTen]
print(len(conTen),len(conIdx))

print(np.real_if_close(ncon(conTen,conIdx)))


<class 'myMPSstuff.myMPS'> <class 'myMPSstuff.myMPS'>
Exp value the hard way: 
0.9997596644683899
Second way: apply the op to the relevant site, then compute the overlap with itself
0.9997596644683899
 Now compute it with the canonical form 
MPS not canonical, bringing to canon form
(8, 8)
(8, 8)
(8, 8, 2)
(8, 8, 2)
(2, 2)
5 5
0.9997596644683877


# build and test environments

In [12]:
o = mpo.myMPO(isi.IsingMPO(LL = 10))

psi = mps.myMPS(mps.randMPS(LL=o.LL))

In [13]:
le, re = envs.build_environments(psi, o)

print(mpomps.expValMPO(psi, o))

for j in range(0,psi.LL):
    #print(np.shape(le[j]), np.shape(re[j]), np.shape(re[j+1]))
    expval = np.real_if_close(ncon([le[j],re[j]],[[1,2,3],[1,2,3]]))
    print(j, expval)


-3.894758112696947
0 -3.8947581126969455
1 -3.8947581126969464
2 -3.8947581126969446
3 -3.894758112696946
4 -3.8947581126969464
5 -3.894758112696946
6 -3.8947581126969446
7 -3.894758112696946
8 -3.8947581126969455
9 -3.894758112696946
